In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import re

In [14]:
raw_data = (pd.read_excel('data/Tonsillectomy_OUTCOME_Data_KQ5_Master.xlsx', 
              sheetname='Outcome data', 
              na_values=['null', 'ND'])
            .drop(['Comments', 'Comments 2', 'Other stats \nName','Other Stats','Results'], 
                  axis=1)
            .rename(columns={'OUTC_Main_\nCATG':'outcome_cat',
                            "Outc_SUB_\nCATG":'outcome_subcat',
                            'Outcome\nN': 'N',
                            'Outcome  \n%': 'outcome_pct',
                            "Outcome\n Mean": 'outcome_mean',
                            "Outcome \nSD": 'outcome_sd',
                            "Outcome \n Median": 'outcome_med',
                            "Outcome \n 95% L": 'outcome_lo_95',
                            "Outcome \n 95% H": 'outcome_hi_95'}))
raw_data.shape

(401, 42)

In [15]:
raw_data.head()

,Citation,Family,Refid,Number of Arms,Rx Grouping,Group_Desc,Dose,Route,Rx_Durn,Last Assesment tmpt for the study,...,outcome_mean,outcome_sd,Outcome SE,outcome_med,Outcome _Q1,Outcome _Q3,Outcome Min,Outcome Max,outcome_lo_95,outcome_hi_95
0,"W. Gao, Q. R. Zhang, L. Jiang and J. Y. Geng. ...",NaN,192,3,G1:,intravenous dexamethasone (DEX-IV),0.5mg/kg,IV,NaN,24 hours post-op,...,234,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"W. Gao, Q. R. Zhang, L. Jiang and J. Y. Geng. ...",NaN,192,3,G2:,local infiltration dexamethasone (DEX-INF),0.5mg/kg,Instillation/infiltration,NaN,24 hours post-op,...,192,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"W. Gao, Q. R. Zhang, L. Jiang and J. Y. Geng. ...",NaN,192,3,G3:,no dexamethasone,NaN,NaN,NaN,24 hours post-op,...,289,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"W. Gao, Q. R. Zhang, L. Jiang and J. Y. Geng. ...",NaN,192,3,G1:,intravenous dexamethasone (DEX-IV),0.5mg/kg,IV,NaN,24 hours post-op,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"W. Gao, Q. R. Zhang, L. Jiang and J. Y. Geng. ...",NaN,192,3,G2:,local infiltration dexamethasone (DEX-INF),0.5mg/kg,Instillation/infiltration,NaN,24 hours post-op,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
intervention_cats = pd.read_excel('data/Tonsillectomy_OUTCOME_Data_KQ5_Master.xlsx', 
              sheetname='KQ5_Relevant_categories')
intervention_cats['Maj_catg'] = intervention_cats.Maj_catg.str.strip()

In [29]:
baseline_data = pd.read_excel('data/Tonsillectomy_OUTCOME_Data_KQ5_Master.xlsx', 
              sheetname='Basic_N', na_values=['null', 'ND', 'NA'])

In [17]:
# Lower case
groups = raw_data.Group_Desc.str.lower()
# Strip information after commas
groups = groups.str.split(',').apply(lambda s: s[0])
# Remove chunks with numeric characters or 'kg', as these are dosages
groups = groups.str.split(' ').apply(lambda s: 
                        ' '.join([si for si in s 
                                  if not re.compile('[\d()/]').search(si)]))
# Combine saline, control and placebo, assume groups starting with 'no' means placebo
groups = groups.replace({'saline':'placebo',
                        'control':'placebo'}).apply(lambda s: 'placebo'*s.startswith('no ') or s)

In [18]:
groups.value_counts()

placebo                                                                 90
dexamethasone                                                           87
ramosetron                                                              27
ondansetron                                                              9
granisetron                                                              9
ibuprofen                                                                8
tropisetron                                                              7
metoclopramide                                                           7
dexamethasone sodium phosphate                                           6
postoperative ketoprofen + saline                                        6
dolasetron                                                               6
intravenous dexamethasone                                                6
levobupivacaine with epinephrine                                         6
diclofenac               

In [30]:
data_merged = raw_data.merge(intervention_cats, on='Refid').merge(baseline_data, on='Refid')

In [31]:
data_merged.Maj_catg.value_counts()

perioperative steroid                  208
perioperative NSAID                     97
perioperative antiemetic                80
perioperative antiemetic or steroid     14
postoperative analgesic                  2
Name: Maj_catg, dtype: int64

Extract top-3 interventions

In [32]:
interventions_of_interest = data_merged.Maj_catg.value_counts()[:3].index.values

In [33]:
data_merged.outcome_cat.value_counts()

Health Care Utilization    216
Harms                      132
Pain management             41
Return to normal diet       12
Name: outcome_cat, dtype: int64

In [23]:
data_merged.outcome_subcat.value_counts()

HC utilization-Need for Rescue meds         105
HC utilization-Additional meds use/ dose     65
Pain management                              41
Harms: Post-op bleeding                      39
HC utilization-Number of Rescue meds         31
Harms-other                                  22
Harms-Re-operation for bleeding              17
HC utilization-# of antibiotics              17
Harms- readmission for bleeding              15
Time to Return to normal diet                12
Harms- readmission for PONV                   9
Harms- readmission for dehydration            6
Harms- readmission for post-op pain           4
Harms-readmission-unspecified                 4
Harms- ER visit for PONV                      3
Harms-Hospital admission                      3
Harms- ER visit-Unspecified                   2
Harms- ER visit for dehydration               2
Harms- ER visit for post-op pain              2
Harms-Death-30 day                            2
Name: outcome_subcat, dtype: int64

Filter readmission data by looking for outcomes with "admission" or "visit" in the name

In [24]:
readmission_outcomes = data_merged[data_merged.outcome_subcat.str.contains(r'admission|visit')]

In [26]:
readmission_outcomes[['N', 'outcome_pct']]

,N,outcome_pct
12,0,0.0
13,0,0.0
69,0,0.0
70,0,0.0
185,1,2.0
186,0,0.0
187,1,2.0
188,1,2.0
189,0,0.0
190,0,0.0
